<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Lab_1_Small_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μικρό Dataset (S11 - Quality Assessment of Digital Colposcopies)


---

#Section A

---


Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8



# Section B

---


Αρχικά ενημερώνουμε τις βιβλιοθήκες που θα χρησιμοποιήσουμε.



In [0]:
# -*- coding: utf-8 -*-
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package
import warnings 
warnings.filterwarnings('ignore')

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)


Κατεβάζουμε το αρχείο που περιέχει το dataset:

In [0]:
from urllib.request import urlretrieve
import os
import zipfile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")


In [0]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00384/Quality%20Assessment%20-%20Digital%20Colposcopy.zip","QADC.zip")
print("All the files are downloaded")

Βλέπουμε ότι το αρχείο όντως κατέβηκε:

In [0]:
!ls

Επειδή είναι compressed, πρεπει να το κάνουμε decompress και να δουμε ότι όντως έγινε:

In [0]:
!unzip -q "QADC.zip"

In [0]:
!ls

Μετονομάζουμε τον φάκελο για να έχει μικρότερο όνομα:



In [0]:
!mv Quality\ Assessment\ -\ Digital\ Colposcopy QADC

Ανοίγοντας τον φάκελο, είδαμε ότι έχουμε τρια αρχεία, τα οποία έχουν headers, και πρέπει 1) να ενοποιηθούν σε ένα αρχείο και 2) να αφαιρεθούν οι επιπρόσθετοι headers. 

In [0]:
with open("QADC/hinselmann.csv",'r') as f:
    with open("QADC/hinselmann1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [0]:
with open("QADC/schiller.csv",'r') as f:
    with open("QADC/schiller1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

Φτιάχνουμε ένα μεγάλο αρχείο:

In [0]:
!cat QADC/green.csv QADC/hinselmann1.csv QADC/schiller1.csv > all.csv

Για να δούμε λοιπόν το dataset μας ολόκληρο:

In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("all.csv")
df



1.   Το dataset περιγράφει μετρήσεις κολποσκοπήσεων και την κατάσταση (ετικέτα) των αντίστοιχων κόλπων, όπως προκύπτει από τις εκτιμήσεις καθενός από 6 ειδικούς και την εκτίμηση της πλειοψηφίας. Οι μετρήσεις και οι εκτιμήσεις είναι διαφορετικές για τις διαφορετικές μεθόδους με τις οποίες πραγματοποιούνται οι κολποσκοπήσεις: Hinselmann, Green, Schiller.



2.    Έχουμε συνολικά 287 δείγματα, 92 από τη μέθοδο Schiller, 98 από τη μέθοδο Green και 97 από τη μέθοδο Hinselmann. Σε κάθε δείγμα περιέχονται μετρήσεις για 62 χαρακτηριστικά-παρατηρήσεις από τα οποία προέκυψαν οι εκτιμήσεις. Όλα τα χαρακτηριστικά είναι διατεταγμένα.Οι μετρήσεις είναι αριθμητικά δεδομένα και αφορούν τιμές όπως τις επιφάνειες περιοχών του κόπλου.

3.    Υπάρχουν επικεφαλίδες στην πρώτη γραμμή πάνω από τα χαρακτηριστικά και τις ετικέτες, οι οποίες θα πρέπει να αφαιρεθούν. Δεν υπάρχει στήλη για την αρίθμηση των γραμμών.

4.    Η τιμή της κατάστασης μπορεί να πάρει δύο τιμές, 0 για κακή και 1 για καλή. Όπως υποδεικνύεται από τις FAQ το πρόβλημα θα αναλυθεί ως binary classification λαμβάνοντας υπόψην μόνο τις εκτιμήσεις της πλειοψηφίας. Έτσι οι στήλες των ετικετών που αφορούν μεμονωμένα τον κάθε ειδικό αφαιρούνται και μένει μόνο η τελευταία στήλη με τις εκτιμήσεις της πλειοψηφίας. Η στήλη αυτή είναι η τελευταία (θέση 69 στον αρχικό πίνακα). Παρατηρούμε οτι όλες οι ισοψηφίες (3-3) επιλύονται θεωρώντας την κατάσταση ως καλή, γεγονός που ίσως μας αναγκάσει να αυξήσουμε αργότερα το πλήθος των δειγμάτων με ετικέτα 0 ή να διαγράψουμε δείγματα με ετικέτα 1.

    
5.    Συνενώνουμε τα αρχεία που αφορούν τις 3 διαφορετικές μεθόδους. Μετά τη συνένωση οι επικεφαλίδες προστέθηκαν ως γραμμές, με αποτέλεσμα να έχουμε 3 φορές την ίδια γραμμή με τις επικεφαλίδες, τις οποίες και αφαιρούμε.

6.    Δεν υπάρχουν απουσιάζουσες τιμές.



Ας τυπώσουμε τον αριθμό, τα ονόματα και τους τύπους του κάθε attribute, για να τα εξετάσουμε καλύτερα:



In [0]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print('{:<10}{:<40}{:<10}{:<20}'.format(str(i+1), str(df.columns[i]),"type: ", str(df.dtypes[df.columns[i]])))


Βλέπουμε ότι έχουμε 5 experts, αλλά εμείς θα δουλέψουμε μόνο με την συνολική άποψη:

In [0]:
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)
df

Ας μετρήσουμε τώρα, πόσοι είναι υγιείς και πόσοι όχι, και θα το αποθηκεύσουμε αυτό σαν τα labels μας (βεβαίως, θα πρέπει μετά να αφαιρέσουμε αυτήν την κολώνα χαρακτηριστικών, πριν κάνουμε training).


In [0]:
labels_df = df.iloc[:, [62]]
labels = labels_df.values.reshape(287,)
positive = 0
negative = 0
for i in range(0, len(labels)):
    positive += labels[i]==1
    negative += labels[i]==0
print("positive = ", positive, " negative: ", negative)

Βλεπουμε οτι εχουμε unbalanced data set, καθως η κλαση των positive(υγειων) ειναι κατα πολυ μεγαλυτερη σε μεγεθος απο την κλαση των negative. (75-25)

Θα πρέπει αρχικά να κάνουμε split τα δεδομένα μας, όπως λέει η εκφώνηση 80-20. 

In [0]:
from sklearn.model_selection import train_test_split


train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2, random_state=78)
print(len(train))


Ας επαληθεύσουμε ότι πάλι ο λόγος των positive(υγειων) με των negative(άρρωστων) είναι πάλι skewed:



In [0]:
positive = 0
negative = 0
for i in range(0, len(train_labels)):
    positive += train_labels[i]==1
    negative += train_labels[i]==0
print("positive = ", positive, " negative: ", negative)

Ας γράψουμε τώρα τις συναρτήσεις μετασχηματισμών και δημιουργίας των classifiers, που θα τις χρησιμοποιήσουμε εντός του pipeline. Έχουμε γράψει συναρτήσεις για selection δεδομένων, resampling (κάτι που θα χρειαστεί σίγουρα, καθώς δεν έχουμε ισορροπημένο dataset), standardization και τέλος για την δημιοργία των κατηγοριοποιητών. Όλες αυτές οι μέθοδοι είναι αρκετά γενικευμένοι και μπορούν να πάρουν σαν παραμέτρους διαφορετικές τεχνικές που ίσως να θέλουμε να εφαρμόσουμε στην κάθε περίπτωση.

In [0]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

"""
    This is a function that applies a transformation (selection) on a dataset, 
    based on the type of the selector. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no transformation happens. The selector argument
    is a string and can be one of two things: either "PCA" or 
    "Variance Threshold". In either case, there can be arguments, and in "PCA"
    the arguments can be an int of the number of principal components, and in
    "Variance Threshold", it can be a float. Returns the transformed dataset.
"""
def Apply_selector(data, selector = None, arguments = None):
    (train, train_labels, test, test_labels) = data
    
    if selector == "PCA":    
        if arguments:
            number_of_components = arguments
            pca = PCA(n_components = number_of_components)
            pca.fit(train)
            train_reduced = pca.transform(train)
            test_reduced = pca.transform(test)
        else:
            pca = PCA()
            pca.fit(train)
            train_reduced = pca.transform(train)
            test_reduced = pca.transform(test)
    elif selector == "Variance Threshold":
        if arguments:
            t = arguments
            sel = VarianceThreshold(threshold=t)
            train_reduced = sel.fit_transform(train)
            test_reduced = sel.transform(test)
        else:
            sel = VarianceThreshold()
            train_reduced = sel.fit_transform(train)
            test_reduced = sel.transform(test)
    else:
        (train_reduced, train_labels, test_reduced, test_labels) = data
    return (train_reduced, train_labels, test_reduced, test_labels)

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

"""
    This is a function that applies resampling on a dataset, based on the 
    type of sampling method. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no sampling happens. The sampling argument
    is a string and can be one of three things: "Over", "Under" or 
    "OverUnder". In any case, there can be arguments, and in Under or Over or
    OverUnder it can be a float with the ratio of samples between classes. 
    Returns the resampled dataset.
"""
def Apply_sampling(data, sampling = None, arguments = None):
    (train, train_labels, test, test_labels) = data
    if sampling == "Over":    
        if arguments:
            ratio = arguments
            ros = RandomOverSampler(sampling_strategy=ratio)
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
        else:
            ros = RandomOverSampler()
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
        return (train_oversampled, train_labels_oversampled, test, test_labels)
    elif sampling == "Under":
        if arguments:
            ratio = arguments
            rus = RandomUnderSampler(sampling_strategy=ratio)
            train_undersampled, train_labels_undersampled = rus.fit_sample(train,train_labels)
        else:
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train,train_labels)
        return (train_undersampled, train_labels_undersampled, test, test_labels)
    elif sampling=="OverUnder":
        if arguments:    
            ratio = arguments
            ros = RandomOverSampler(sampling_strategy=ratio)
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train_oversampled,train_labels_oversampled)
        else:
            ros = RandomOverSampler()
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train_oversampled,train_labels_oversampled)
        return (train_undersampled, train_labels_undersampled, test, test_labels)
    else:
        return data

In [0]:
from scipy import stats as st

"""
    This is a function that applies a transformation (standardization) on a dataset, 
    based on the type of the standardizer. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no transformation happens. The selector argument
    is a string and can be one of two things: either "zscore" or 
    "minmax". Argument arguments is always None. Returns the transformed dataset.
"""
def Apply_standardizer(data, standardizer = None,  arguments = None):
    (train, train_labels, test, test_labels) = data
    if standardizer=="zscore":
        std_train = st.zscore(train)
    elif standardizer=="minmax":
        std_train = (train - np.min(train) )/ (np.max(train) - np.min(train))
    else:
        std_train = train
    return (std_train, train_labels, test, test_labels)

In [0]:
from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier

"""
    This is a function that creates a classifier object. It does not train, 
    it takes the name of the classifier, along with the arguments we want 
    to pass to the instance creator for the classifier and returns the 
    classifier object.
"""
def Create_classifier(classifier,  arguments = None):
    if classifier=="kNN":
        if arguments and arguments!=-1:
            clf = neighbors.KNeighborsClassifier(n_jobs = -1, n_neighbors = arguments)
        else:
            clf = neighbors.KNeighborsClassifier(n_jobs = -1)
    else:
        if arguments:
            strat = arguments
            clf = DummyClassifier(strategy=strat)
        else:
            clf = neighbors.KNeighborsClassifier()
    return clf


Εδώ ορίζουμε την pipeline, η οποία αντιστοιχίζει τα steps της προεπεξεργασίας δεδομένων και της δημιοργίας των classifiers με τις παραπάνω μεθόδους. 

In [0]:
"""
    This is an implementation for the function imblearn.pipeline.Pipeline().
    The way this function works is, it takes as input two arguments. The first
    is a list. This list contains the steps that need to be executed in the 
    pipeline, IN ORDER. Always the last step is the classifier. Each step is
    a tuple, which contains the name of the step, and the arguments needed to 
    execute the step, like so: ("name", arguments). The second argument of the 
    function is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The function outputs a
    tuple, which contains the processed data, ready for fitting the classier, 
    and the classifier object, as specified by the arguments, like so:
    returns: (processed_data, classifier_object)
"""
def Pipeline(steps, data):
    steps_dict = {"selector": Apply_selector,
                  "sampling": Apply_sampling,
                  "standardizer": Apply_standardizer,
                  "kNN": Create_classifier,
                  "dummy": Create_classifier}
    for step in steps:
        if step[0]!="kNN" and step[0]!="dummy":
            data = steps_dict[step[0]](data,step[1])
        else:
            return ( data, Create_classifier(step[0], arguments = step[1]) )


Η συνάρτηση model_tuning ουσιαστικά υλοποιεί την gridsearchcv, και δημιουργεί για κάθε παράμετρο, για κάθε αλληλουχία steps, για κάθε dataset ένα pipe, και στην συνέχεια κάνει train τα δεδομένα και βγάζει ένα training accuracy. 

In [0]:
import time
from sklearn.metrics import accuracy_score

"""
    This is an implementation for the function gridsearchcv. It takes 3
    arguments, a tuple (model_name, model_parameters), the data to be fitted
    and trained, and the specific parameters. The data parameter needs to be 
    in the form: (train_data, train_labels, test_data, test_labels). Lastly, 
    the Steps argument is a list, that has the steps for the data preprocessing,
    as described in the Pipeline Function. For this exercise, we are dealing 
    only with dummy classifiers and kNN. The model_parameters if not None, is 
    a list of parameters (strings for dummy, ints for kNN).
    Returns (classifier_object, best_parameter, accuracy_list)
"""
def model_tuning(model, data, preprocessing_steps = None):
    #Start measuring the time:
    start_time = time.time()
    #if the classifier is a dummy:
    if model[0]=="dummy":
        #and it has parameters (strategy):
        if model[1]!=None:


            #initialize the lists with the possible best parameters, and the average
            #accuracies for each of the model parameter
            avg_accuracy_list = []
            best_parameter = model[1][0]
            #for each parameter, run the 
            for parameter in model[1]:
    
                #if there are preprocessing steps, add the model_creation 
                #to the pipeline:
                if preprocessing_steps:
                    steps = preprocessing_steps + [(model[0], parameter)]
                else:
                    steps = [(model[0], parameter)]
                #create the pipeline, returns the processed data, the 
                #classifier ready for training and evaluating:
                folded_data = ten_fold(data)

                #initialize a list with the accuracies for the folded data:
                accuracy_list = []
                #train for each of the 10 folds, begin from scratch every time,
                #and then average out the results.
                for data in folded_data:
                    
                    #process the data, create the classifier
                    (processed_data, clf) = Pipeline(steps, data)
                    (train, train_labels, test, test_labels) = processed_data
                    #fit the classifier
                    clf = clf.fit(train, train_labels)
                    #make the predictions based on the training
                    preds = clf.predict(test)
                    accuracy_list.append(accuracy_score(test_labels, preds))
                
                
                
                #store the average accuracy
                avg_accuracy_list.append(sum(accuracy_list)/len(accuracy_list))
                
                #if the average accuracy of the 10 folds is the best one yet, 
                #keep the parameter
                if (sum(accuracy_list)/len(accuracy_list)) >= max(avg_accuracy_list):
                    best_parameter = parameter
        
        
        
        #if the dummy classifier has no parameters:                    
        else:
                #initialize the lists with the possible best parameters, and the average
                #accuracies for each of the model parameter
                avg_accuracy_list = []

                #if there are preprocessing steps, add the model_creation 
                #to the pipeline:
                if preprocessing_steps:
                    steps = preprocessing_steps + [(model[0], model[1])]
                else:
                    steps = [(model[0], model[1])]
                #create the pipeline, returns the processed data, the 
                #classifier ready for training and evaluating:
                folded_data = ten_fold(data)
                
                #initialize a list with the accuracies for the folded data:
                accuracy_list = []

                #train for each of the 10 folds, begin from scratch every time,
                #and then average out the results.
                for data in folded_data:
                    
                    #process the data, create the classifier
                    (processed_data, clf) = Pipeline(steps, data)
                    (train, train_labels, test, test_labels) = processed_data
                    #fit the classifier
                    clf = clf.fit(train, train_labels)
                    #make the predictions based on the training
                    preds = clf.predict(test)
                    accuracy_list.append(accuracy_score(test_labels, preds))
                
                #since no parameter list has been given, best_param = param
                best_parameter = None
                #store the average accuracy
                avg_accuracy_list = sum(accuracy_list)/len(accuracy_list)
    
    
    #if the classifier is a kNN:
    elif model[0] == "kNN":

        #if the kNN classifier has no parameters:    
        if model[1]== -1:
            #initialize the lists with the possible best parameters, and the average
            #accuracies for each of the model parameter
            avg_accuracy_list = []

            #if there are preprocessing steps, add the model_creation 
            #to the pipeline:
            if preprocessing_steps:
                steps = preprocessing_steps + [(model[0], model[1])]
            else:
                steps = [(model[0], parameter)]
            #create the pipeline, returns the processed data, the 
            #classifier ready for training and evaluating:
            folded_data = ten_fold(data)
            
            #initialize a list with the accuracies for the folded data:
            accuracy_list = []
                
            #train for each of the 10 folds, begin from scratch every time,
            #and then average out the results.
            for data in folded_data:
                
                #process the data, create the classifier
                (processed_data, clf) = Pipeline(steps, data)
                (train, train_labels, test, test_labels) = processed_data
                #fit the classifier
                clf = clf.fit(train, train_labels)
                #make the predictions based on the training
                preds = clf.predict(test)
                accuracy_list.append(accuracy_score(test_labels, preds))
            
            #since no parameter list has been given, best_param = None
            best_parameter = None
            #store the average accuracy
            avg_accuracy_list = sum(accuracy_list)/len(accuracy_list)

        #if it has parameters (number_of_neighbors):    
        else:
            #initialize the lists with the possible best parameters, and the average
            #accuracies for each of the model parameter
            avg_accuracy_list = []
            best_parameter = model[1][0]
            #for each parameter, run the 
            for parameter in model[1]:
    
                #if there are preprocessing steps, add the model_creation 
                #to the pipeline:
                if preprocessing_steps:
                    steps = preprocessing_steps + [(model[0], parameter)]
                else:
                    steps = [(model[0], parameter)]
                #create the pipeline, returns the processed data, the 
                #classifier ready for training and evaluating:
                folded_data = ten_fold(data)
                
                #initialize a list with the accuracies for the folded data:
                accuracy_list = []

                #train for each of the 10 folds, begin from scratch every time,
                #and then average out the results.
                for data in folded_data:
                    
                    #process the data, create the classifier
                    (processed_data, clf) = Pipeline(steps, data)
                    (train, train_labels, test, test_labels) = processed_data
                    #fit the classifier
                    clf = clf.fit(train, train_labels)
                    #make the predictions based on the training
                    preds = clf.predict(test)
                    accuracy_list.append(accuracy_score(test_labels, preds))
                print(accuracy_list)
                #store the average accuracy
                avg_accuracy_list.append(sum(accuracy_list)/len(accuracy_list))


                #if the average accuracy of the 10 folds is the best one yet, 
                #keep the parameter
                if (sum(accuracy_list)/len(accuracy_list)) >= max(avg_accuracy_list):
                    best_parameter = parameter
                
        
        

    print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
    return (clf, best_parameter, avg_accuracy_list)

Η συνάρτηση αυτή είναι η υλοποίηση της Kfold(), και σπάει σε διπλώματα το dataset. Συγκεκριμένα, το σπάει σε 10 διαφορετικά dataset, προκειμένου να αξιοποιήσουμε καλύτερα τον σχετικά μικρό όγκο δεδομένων μας. 

In [0]:
from sklearn.utils import shuffle

"""This function is an implementation for KFold(). It takes one argument that 
is a tuple in the form (train, train_labels, test, test_labels), and returns
a list of k (10) tuples in the same form. It shuffles the train and train 
labels, to randomize the data, before splitting. """
def ten_fold(data):
    #In our exercise, the number of folds is 10:
    k = 10
    #Reading the data, no need to read the test and test_labels data.
    train, train_labels, _, _ = data
    #Randomize the data:
    train, train_labels = shuffle(train, train_labels)
    #train is a pandas dataframe, need to make it a numpy array:
    train = np.asarray(train)
    #This is the size of the test set each time:
    fold_size = int(len(train)/k)
    #Create the list to be returned, fill it:
    folds = []
    for i in range(0,10):
        train1 = []
        train_labels1 = []
        test = []
        test_labels = []
        for j in range(0, len(train)):
            if j>= i*fold_size and j<(i+1)*fold_size:
                test.append(train[j])
                test_labels.append(train_labels[j])
            else:
                train1.append(train[j])
                train_labels1.append(train_labels[j])
        folds.append((train1, train_labels1, test, test_labels))
    return folds

Ας ξαναδιαβάσουμε από την αρχή τα δεδομένα μας, και ας τα χωρίσουμε πάλι σε training data και test data πριν προχωρήσουμε στο Γ. Πρέπει όμως πριν, να αφαιρέσουμε την στήλη που περιέχει τις ετικέτες για την υγεία των ασθενών.

In [0]:
df = pd.read_csv("all.csv")
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)
labels_df = df.iloc[:, [62]]
labels = labels_df.values.reshape(287,)


df = df.drop(df.columns[62], axis=1)
train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2, random_state=78)


#Γ 
---

Για το section αυτό, αρχικά πρέπει να εκπαιδεύσουμε τους classifiers (dummy και kNN) στο train dataset, με deafult τιμές. Καλόυμε την συνάρτηση model_tuning, και για τα δύο μοντέλα, χωρίς επιπλέον παραμέτρους, και θα λάβουμε πίσω τους classifiers, την καλύτερη παράμετρο (εδώ None, αφού δεν τους έχουμε δώσει κάποια παράμετρο για training) και το average training accuracy πάνω σε 10 folds. 

Δεν θα κάνουμε κάποια preprocessing steps. 

In [0]:
#Group the splitted dataset into a tuple:
data = (train, train_labels, test,  test_labels)

dummy_default, best_parameter1, acc_list1 = model_tuning( ("dummy", None),  data)  
kNN_default, best_parameter2, acc_list2 = model_tuning( ("kNN", -1),  data)  

print("The average training accuracy of the default dummy classifier is: ", acc_list1)
print("The average training accuracy of the default kNN classifier is: ", acc_list2)

Τώρα θα κάνουμε εκτίμηση στο test set και για τους δύο, θα βρούμε το confusion matrix τους, και θα τυπώσουμε το f1-micro και f1-macro average:

In [0]:




print("best parameter = ",best_parameter,"\taccuracy: ", acc_list)
print("2:")
clf, best_parameter, acc_list = model_tuning( ("dummy", None),  data, preprocessing_steps= [("selector", "PCA"),("sampling","Under",0.5), ("standardizer", "zscore")]   )    
print("best parameter = ",best_parameter,"\taccuracy: ", acc_list)

print("3:")
acc_list = []
data = (train, train_labels, test,  test_labels)
near = [i for i in range(1,22)]
model = ("kNN", near)
clf, best_parameter, acc_list = model_tuning(model, data)
print("best parameter = ",best_parameter ,"\tmax_accuracy: ", max(acc_list))
print(acc_list)
print(acc_list)